In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/home/sergio/Downloads/semeion.data', sep=' ', header=None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1593 entries, 0 to 1592
Columns: 267 entries, 0 to 266
dtypes: float64(257), int64(10)
memory usage: 3.2 MB


In [3]:
labels = 1*df.iloc[:,257]+2*df.iloc[:,258]+3*df.iloc[:,259]+4*df.iloc[:,260]+5*df.iloc[:,261]+6*df.iloc[:,262]+7*df.iloc[:,263]+8*df.iloc[:,264]+9*df.iloc[:,265]

In [4]:
from sklearn.cross_validation import train_test_split
from conformal_predictors.nc_measures.SVM import SVCDistanceNCMeasure
from conformal_predictors.predictors.SVC import ConformalSVC
from conformal_predictors.predictors.SVC import ConformalNuSVC
from sklearn.decomposition import PCA

data = df.iloc[:, range(0,256)].values.astype(int)
pca = PCA(n_components=10)
data = pca.fit_transform(data)
print(data)
nc = SVCDistanceNCMeasure()
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)
X_proper_train, X_calibration, y_proper_train, y_calibration = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
nc = SVCDistanceNCMeasure()
# C=1000.0, kernel='rbf', gamma=0.01
cp = ConformalSVC(nc, C=100.0, kernel='rbf', gamma=0.1)
cp.fit(X_proper_train, y_proper_train)
print('B')
print(cp._clf.decision_function(X_proper_train))
print(y_proper_train)
print('S')
cp.calibrate(X_calibration, y_calibration)
y_pred, credibility, confidence = cp.predict_cf(X_test)
y_pred_2 = cp._clf.predict(X_test)
print(cp._cal_a)



[[ 0.70055964 -1.90949693 -0.03808341 ...,  2.35052543 -0.47125119
   1.86686731]
 [ 4.48208702  0.80313984  1.7303013  ..., -0.25508896  0.40252754
  -0.20714367]
 [ 2.55710444 -2.43951127  0.68400161 ..., -0.47858184 -0.09134766
  -1.17951301]
 ..., 
 [-0.45726971  4.11623298  0.15029695 ...,  0.30735726 -0.52209076
   0.39034959]
 [-0.73962227  1.08841281  2.50412925 ...,  0.78317408 -1.78593514
  -0.73518968]
 [-1.14483951 -1.40051338  1.4924586  ..., -0.88884965  0.06619578
   0.69018879]]
B
[[ 8.64525769  2.08458324  3.04923091 ...,  0.1626798   5.05581079
   6.94974602]
 [ 0.10142762  4.0549151   8.68840567 ...,  3.03974074  7.94046421
   6.00165423]
 [ 6.90765065  2.09926821  5.0146226  ...,  1.11162228  3.12495823
   4.07852184]
 ..., 
 [ 1.09311049  3.0467591   4.03517987 ...,  0.143618    7.0056095
   7.88777137]
 [ 2.06503763  4.03718501  6.99665484 ...,  1.08653135  8.70462475
   7.91551076]
 [ 1.10597888  4.06859318  6.93363854 ...,  0.13510176  8.69264399
   5.93830587]]

TypeError: list indices must be integers, not tuple

In [ ]:
from sklearn.metrics import precision_score, accuracy_score, f1_score, roc_auc_score

print(precision_score(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

print(precision_score(y_test, y_pred_2))
print(accuracy_score(y_test, y_pred_2))

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

idx = np.ix_(y_pred != y_test)[0]
idx_2 = np.ix_(y_pred_2 != y_test)[0]
idx_3 = np.ix_(y_pred != y_pred_2)[0]

plt.plot(range(0,len(credibility)), credibility, 'ob')
plt.plot(np.array(range(0,len(credibility)))[idx], credibility[idx], 'or')
#plt.plot(np.array(range(0,len(credibility)))[idx_2], credibility[idx_2], 'og')
plt.axis('tight')
plt.ylim([-0.05, 1.05])

In [ ]:
y_pred_3, credibility_3, confidence_3 = cp.predict_cf(X_test[idx_3, :])
print(y_test[idx_3])

In [ ]:
idx_4 =  np.ix_((y_pred != y_test) & (credibility > 0.8))[0]
print(credibility[idx_4])
print(len(idx_4))
y_pred_4, credibility_4, confidence_4 = cp.predict_cf(X_test[idx_4, :])
print(credibility_4)
y_pred_4, credibility_4, confidence_4 = cp.predict_cf(np.array([X_test[idx_4[-1], :]]))
print(credibility_4)
y_pred_4, credibility_4, confidence_4 = cp.predict_cf(np.array(X_test[idx_4[[-2,-1]], :]))
print(credibility_4)
y_pred_4, credibility_4, confidence_4 = cp.predict_cf(np.array([X_test[idx_4[-1], :]]))
y_pred_4, credibility_4, confidence_4 = cp.predict_cf(np.array(X_test[idx_4[[-2,-1]], :]))

In [ ]:
clf = cp._clf

In [ ]:
np.set_printoptions(precision=5)
clf.decision_function(X_test[0,:].reshape(1, -1))

In [ ]:
clf.decision_function(X_test[[0,1],:])

In [ ]:
clf.decision_function(X_test[0,:].reshape(1, -1))

In [ ]:
import numpy as np
X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y = np.array([1, 1, 2, 2])
from sklearn.svm import SVC
clf = SVC()
clf.fit(X, y) 
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
print(clf.predict([[-0.8, -1]]))

In [ ]:
clf.decision_function(X[0,:].reshape(1, -1))
clf.decision_function(X[1,:].reshape(1, -1))

In [ ]:
clf.decision_function(X[[0,1],:])